# mongodb
1. 라이브러리 설치
2. 라이브러리 로드
3. CRUD
    - create : insertOne() 또는 insertMany()
    - read : find()
    - update : updateOne(), updateMany(), replaceOne() 등
    -  delete : deleteOne(), deleteMany()
4. 터미널에 brew services start mongodb-community 를 쳐야 시작됨

In [1]:
!pip3 install pymongo

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 413 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 283 kB 24.4 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from pymongo import MongoClient

In [8]:
client = MongoClient(
    host = 'localhost',
    port = 27017
)

In [19]:
# mongodb에 있는 데이터베이스의 목록 출력
client.list_database_names()

['admin', 'config', 'local', 'ubion']

In [10]:
# 데이터베이스를 선택
db = client['ubion']

In [11]:
# collection 접근 (collection은 sql에서 table과 같은 의미)
collection = db['test']

In [12]:
# CRUD -> C
# create -> insert()
# insert_one() / insert_many()
data ={
    'id' : 'test',
    'pass' : '1234'
}
collection.insert_one(data)

In [14]:
collection.find_one()
# 이거 결과값의 첫번째꺼가 프라이머리 키로 자동 인식됨

{'_id': ObjectId('643f8af75a0ff4f119cc82c5'), 'id': 'test', 'pass': '1234'}

In [17]:
data = [
    {
    'name' : 'test1',
    'phone' : '010111122222',
    'gender' : 'male'
    },
    {
    'loc' :'seoul'
    }
]
collection.insert_many(data)

In [18]:
list(collection.find())

[{'_id': ObjectId('643f8af75a0ff4f119cc82c5'), 'id': 'test', 'pass': '1234'},
 {'_id': ObjectId('643f8d5f5a0ff4f119cc82c6'),
  'name': 'test1',
  'phone': '010111122222',
  'gender': 'male'},
 {'_id': ObjectId('643f8d5f5a0ff4f119cc82c7'), 'loc': 'seoul'}]

1. sql에서 name이 test1인 경우의 데이터를 출력
    - select * from test where name = 'test1'

In [20]:
# 위에 있는 쿼리를 똑같이 만든거
list(collection.find({'name' : 'test1'}))

[{'_id': ObjectId('643f8d5f5a0ff4f119cc82c6'),
  'name': 'test1',
  'phone': '010111122222',
  'gender': 'male'}]

1. 해당하는 컬럼의 데이터만 출력하려면
    - select name from test

In [21]:
list(collection.find({},{'name' : 1}))

[{'_id': ObjectId('643f8af75a0ff4f119cc82c5')},
 {'_id': ObjectId('643f8d5f5a0ff4f119cc82c6'), 'name': 'test1'},
 {'_id': ObjectId('643f8d5f5a0ff4f119cc82c7')}]

1. 데이터의 수정
    - update test set pass ='0000' where id = 'test' (sql)
    - test 테이블에 id가 test인 데이터에서 pass의 값을 0000으로 수정하겠다

In [23]:
collection.update_one(
    {
        'id' : 'test'
    },
    {
        "$set" :{
            'pass' : '0000'
        }
    }
)

In [25]:
collection.find_one()

{'_id': ObjectId('643f8af75a0ff4f119cc82c5'), 'id': 'test', 'pass': '0000'}

1. 데이터의 삭제
    - delete from test where id = 'test'
    - test 테이블에서 id가 'test'인 데이터를 삭제

In [26]:
collection.delete_one(
    {
        'id' : 'test'
    }
)

In [31]:
list(collection.find())

[{'_id': ObjectId('643f8d5f5a0ff4f119cc82c6'),
  'name': 'test1',
  'phone': '010111122222',
  'gender': 'male'},
 {'_id': ObjectId('643f8d5f5a0ff4f119cc82c7'), 'loc': 'seoul'}]

In [35]:
# 위에랑 똑같은데 collection으로 안묶고 하기
list(client['ubion']['test'].find())

[{'_id': ObjectId('643f8d5f5a0ff4f119cc82c6'),
  'name': 'test1',
  'phone': '010111122222',
  'gender': 'male'},
 {'_id': ObjectId('643f8d5f5a0ff4f119cc82c7'), 'loc': 'seoul'}]

### mongodb select 조건식

- find(옵션1, 옵션2, 옵션2, ...., 컬럼 선택)

    1. db.collection.find() -> select * from TABLE
    2. db.collection.find({}, {'id' : 1, 'pass' : 1}) -> select id, pass from TABLE
    3. db.collection.find({}, {'id' : 1, 'pass' : 1, 'id' : 0}) ->  2번과정과 같은 sql 문이지만 _id를 보여주지않겠다
    4. db.collection.find({'id' : 'test'}) -> select * from TABLE where id = 'test'
    5. db.collection.find({'id' : 1, 'pass' : '1234'}) -> select * from TABLE where id = 'test' and pass = '1234'
    6. db.collection.find({'id' : 1}, {'pass' : 1}) -> select * from TABLE where id = 'test' or pass ='1234'
    7. db.collection.find({"age" : {"$gt" : 25}}) -> select * from TABLE where age > 25
    8. db.collection.find({"age" : {"$lt" : 25}}) -> select * from TABLE where age < 25
    9. db.collection.find({"id" : {/bc/}}) -> select * from TABLE where id like '%bc%'
    10. db.collection.find().sort({"id" : 1}) ->select * from TABLE order by id ASC
    11. db.collection.find().sort({"id" : -1}) ->select * from TABLE order by id DESC
    12.  db.collection.find().limit(10) -> select * from TABLE limit 10